# Modelado incremental de FPGAs

Te recomiendo que te leas el artículo que te adjunto. En él se describe en detalle el problema que queremos abordar, la solución que proponemos, y los resultados obtenidos. En particular sería interesante que leyeses la sección 6, _Modeling Strategy_, donde se explica qué modelos se emplean y cuáles son las características y variables a predecir.

## Escenario

El cometido del sistema es ir acelerando tareas provenientes de una _workload_ según estas van llegando. Esto se realiza descargando dichas tareas en la lógica reconfigurable de la FPGA tan pronto llegan, siempre y cuando haya recursos disponibles (las tareas quepan dentro de la lógica disponible en el momento).

Puesto que cada tarea utiliza únicamente una porción de la lógica, se da el caso de que multiples tareas diferentes son aceleradas de forma simultánea dentro de la FPGA.
Esta coexistencia de tareas en la FPGA deriva en una contención en los recursos disponibles (cpu, ram, acceso al bus, etc), provocando que el tiempo de ejecución y el consumo de potencia de una tarea _X_ se vea influenciado por qué otras tarea(s) _Y_ se estén ejecutando en ese instante.

## Propuesta

Lo que nosotros buscamos es poder modelar esta interacción entre tareas, para ser capaces de predecir el consumo y tiempo de ejecución, y luego tomar decisiones de planificación de tareas inteligentes.

Para llevar a cabo esto, hacemos lo siguiente:

1. Mientras que el sistema va ejecutando tareas, proceso que se prolonga de forma indefinida en el tiempo, se van realizando mediciones periódicas de consumo de potencia y tiempo de ejecución (para ello se emplea un componente específico que denominamos _Monitor_).
   
2. Estas mediciones se procesan segun se van obteniendo, generando así un conjunto de observaciones que poder usar para entrenar los modelos. Esto se hace en tiempo de ejecución en el propio procesador de la placa con un módulo de Python.

    Dichas observaciones contienen básicamente el uso de la CPU (hemos observado un impacto importante en el rendimiento del sistema), la configuración particular de la FPGA en dicho instante, y las variables a predecir (puesto que se trata de aprendizaje supervisado). _La `tabla 2`del mencionado paper lista los campos dentro de cada observación._

3. Conforme las observaciones se van generando, estas se envían a los modelos para actualizarlos de forma incremental. Esto se hace nuevamente con un módulo de Python, empleando la biblioteca [river](https://riverml.xyz/latest/) que está específicamente diseñada para aprendizaje incremental.

    El proceso de entrenamiento es algo más complejo, pues intentamos reducir el tiempo de entrenamiento lo máximo posible mediante un mechanismo que gestiona de forma inteligente cuándo deben ser actualizados los modelos (sección 5, _Model Orchestrator for Incremental Learning_, del paper).

4. Este proceso se repite de forma indefinida.

_Todo esto está dentro del este [repositorio](https://github.com/juanea7/fpga-modeling). No está en estado de revista en estos momentos, pero en caso de que tengas curiosidad..._

## Proceso de entrenamiento

El proceso de entrenamiento es algo complejo, pero aquí te muestro una versión simplificada con lo esencial.

### Aspectos omitidos

- No se tiene en cuenta el mecanismo que gestiona el proceso de entrenamiento, simplemente se entrena el modelo con cada observación una a una.
- No se muestra el tratamiento de las trazas para pasar de las mediciones realizadas con el _Monitor_, las observaciones presentes en el dataset estás ya correctamente procesadas.
- Se han omitido todos los procesos relacionados con la gestión del entrenamiento en tiempo de ejecución (temas relacionados con comunicación entre las distintas partes de la infraestructura).

### Dataset

El dataset incluido en esta carpeta es en esencia una `DataFrame` de `Pandas`que contiene cada una de las observaciones generadas durante la ejecución de una workload con el sistema real.

### Leer dataset con `Pickel`

In [1]:
import pandas as pd

dataset_df = pd.read_pickle("dataset.pkl")

print(f"Número de observaciones en el dataset: {len(dataset_df)}")
# Se puede hacer con print, pero así queda más bonitor
display(dataset_df[:3])

Número de observaciones en el dataset: 98525


,user,kernel,idle,Main,aes,bulk,crs,kmp,knn,merge,nw,queue,stencil2d,stencil3d,strided,Top power,Bottom power,Time
0,66.667,11.667,21.667,3,0,0,0,1,0,0,0,0,0,0,0,1.383,0.176,1.121
1,45.902,11.475,42.623,2,0,2,1,0,0,0,0,0,0,0,0,1.442,0.174,1.712
2,45.902,11.475,42.623,1,0,2,1,0,0,0,0,0,0,0,0,1.443,0.174,1.173


Es importante tener en cuenta que actualmente el modelo está diseñado para trabajar con [MachSuite](https://breagen.github.io/MachSuite/) un conjunto de benchamarks basados en tareas de aceleración HW condificados en HLS. Nosotros identificamos cada uno de los kernels disponibles con un ID de la siguiente manera:

| AES | BULK | CRS | KMP | KNN | MERGE | NW | QUEUE | STENCIL2D | STENCIL3D | STRIDED |
| :--------: | :--------: | :--------: | :--------: | :--------: | :--------: | :--------: | :--------: | :--------: | :--------: | :--------: |
| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 |


Se puede ver como cada obsercación consta de:
- Tres características que describen el uso de la CPU (`user`, `kernel`, `idle`).
- Una característica que indica qué tarea está siendo evaluado en base a su ID (`Main`).
- El número de replicas de cada tarea que hay presentes en ese momento en la FPGA (`nombre_de_la_tarea`).
- Una etiqueta por cada variable a predecir (en este caso tres, consumo del PS, consumo del PL y tiempo de ejecución).

### Formatear dataset

Para que sea más cómodo trabajar con él, nosotros dividimos le dataset en `features` y `labels`.

In [2]:
# Extract features
features_df = dataset_df.drop(["Top power", "Bottom power", "Time"], axis=1)

# Extract labels
labels_df = dataset_df[["Top power", "Bottom power", "Time"]]

display(features_df[:3])
display(labels_df[:3])

,user,kernel,idle,Main,aes,bulk,crs,kmp,knn,merge,nw,queue,stencil2d,stencil3d,strided
0,66.667,11.667,21.667,3,0,0,0,1,0,0,0,0,0,0,0
1,45.902,11.475,42.623,2,0,2,1,0,0,0,0,0,0,0,0
2,45.902,11.475,42.623,1,0,2,1,0,0,0,0,0,0,0,0


,Top power,Bottom power,Time
0,1.383,0.176,1.121
1,1.442,0.174,1.712
2,1.443,0.174,1.173


### Inicializar los modelos

Como hemos comentado antes, nosotros usamos unos modelos específicamente diseñados para aprendizaje incremental.

In [3]:
import river
from river import metrics, preprocessing, forest

# PS power model
top_power_model = (
                river.preprocessing.StandardScaler() |
                river.tree.HoeffdingAdaptiveTreeRegressor(
                    max_depth=100,
                    grace_period=50,
                    model_selector_decay=0.05,
                    seed=42
                )
            )

# PL power models
bottom_power_model = (
                river.preprocessing.StandardScaler() |
                river.tree.HoeffdingAdaptiveTreeRegressor(
                    max_depth=100,
                    grace_period=50,
                    model_selector_decay=0.05,
                    seed=42
                )
            )

# Execution time model
time_model = tmp_model = river.forest.ARFRegressor(seed=42, max_features=None, grace_period=50, n_models = 5, max_depth=100, model_selector_decay=0.05)

# Model MAPE metrics
top_power_mape = river.utils.Rolling(river.metrics.MAPE(), window_size=1000)
bottom_power_mape = river.utils.Rolling(river.metrics.MAPE(), window_size=1000)
time_mape = river.utils.Rolling(river.metrics.MAPE(), window_size=1000)

# List of metrics and models
models = [top_power_model, bottom_power_model, time_model]
metrics = [top_power_mape, bottom_power_mape, time_mape]

### Entrenar los modelos con el dataset

Iteramos sobre el dataset, actualizando los modelos con cada una de las observaciones de forma incremental, el proceso es básicamente este:
1. Convertimos las características al tipo de datos que nos conviene con la función `features_labels_accommodation()`. Esta no es la manera óptima. Esa conversión se hará a la hora de generar el dataset, pero ahora mismo tiene que ser así.

In [4]:
def features_labels_accommodation(features, labels):
        """Perform accomodation on features and labels. Type casting..."""

        features["user"] = float(features["user"])
        features["kernel"] = float(features["kernel"])
        features["idle"] = float(features["idle"])

        features["Main"] = int(features["Main"])
        features["aes"] = int(features["aes"])
        features["bulk"] = int(features["bulk"])
        features["crs"] = int(features["crs"])
        features["kmp"] = int(features["kmp"])
        features["knn"] = int(features["knn"])
        features["merge"] = int(features["merge"])
        features["nw"] = int(features["nw"])
        features["queue"] = int(features["queue"])
        features["stencil2d"] = int(features["stencil2d"])
        features["stencil3d"] = int(features["stencil3d"])
        features["strided"] = int(features["strided"])

        # Get each model label
        labels = [float(labels[key]) for key in labels]

        return features, labels

2. Hacemos una predicción con el modelo para la observación actual.

3. Entrenamos el modelo con la observación actual.

4. Actualizamos la metrica de error usando la predicción del modelos y el valor real medido.

Acontinuación se muestra el proceso de entrenamiento. _Se ha hecho solo para las primeras 5,000 observaciones para ahorrar tiempo._

In [ ]:
import time
import numpy as np
import pickle

def rolling(data):
    # Extract the MAPE values from the nested list
    error_data = [point[1] for point in data]
    rolling_mean = np.zeros(len(error_data)-1999)
    rolling_sdv = np.zeros(len(error_data)-1999)

    # Compute the rolling mean for 1000 values
    buffer = error_data[1000:2000]
    rolling_mean[0] = np.mean(buffer)
    rolling_sdv[0] = np.std(buffer)
    j = 0
    for i in range(2000, len(error_data)):
        buffer[j] = error_data[i]
        rolling_mean[i-1999] = np.mean(buffer)
        rolling_sdv[i-1999] = np.std(buffer)
        j += 1
        if j == 1000:
            j = 0
    return rolling_mean, rolling_sdv
    

# Loop over the observations
infer_time_TP = 0
train_time_TP = 0
infer_time_BP = 0
train_time_BP = 0
infer_time_Time = 0
train_time_Time = 0

MAPE_mean_TP = []
MAPE_std_TP = []
MAPE_mean_BP = []
MAPE_std_BP = []
MAPE_mean_Time = []
MAPE_std_Time = []

for i in range(5):
    buffer_MAPE_TP = []
    buffer_MAPE_BP = []
    buffer_MAPE_Time = []
    j = 0
    for features, labels in river.stream.iter_pandas(features_df, labels_df, shuffle=False, seed=42):
        # Features and labels accommodation
        features, labels = features_labels_accommodation(features, labels)
        
        start_time = time.perf_counter()
        # Make a prediction
        y_pred = top_power_model.predict_one(features)
        end_time = time.perf_counter()
        infer_time_TP = infer_time_TP + end_time - start_time
        start_time = time.perf_counter()
        # Train the model
        top_power_model.learn_one(features, labels[0])
        # Update metric
        top_power_mape.update(labels[0], y_pred)
        end_time = time.perf_counter()
        train_time_TP = train_time_TP + end_time - start_time
        error = (labels[0]-y_pred)/labels[0]*100
        loss = [j, error]
        buffer_MAPE_TP.append(loss)
        

        start_time = time.perf_counter()
        # Make a prediction
        y_pred = bottom_power_model.predict_one(features)
        end_time = time.perf_counter()
        infer_time_BP = infer_time_BP + end_time - start_time
        start_time = time.perf_counter()
        # Train the model
        bottom_power_model.learn_one(features, labels[1])
        # Update metric
        bottom_power_mape.update(labels[1], y_pred)
        end_time = time.perf_counter()
        train_time_BP = train_time_BP + end_time - start_time
        error = (labels[1]-y_pred)/labels[1]*100
        loss = [j, error]
        buffer_MAPE_BP.append(loss)
        

        start_time = time.perf_counter()
        # Make a prediction
        y_pred = time_model.predict_one(features)
        end_time = time.perf_counter()
        infer_time_Time = infer_time_Time + end_time - start_time
        start_time = time.perf_counter()
        # Train the model
        time_model.learn_one(features, labels[2])
        # Update metric
        time_mape.update(labels[2], y_pred)
        end_time = time.perf_counter()
        train_time_Time = train_time_Time + end_time - start_time
        error = (labels[2]-y_pred)/labels[2]*100
        loss = [j, error]
        buffer_MAPE_Time.append(loss)
        j += 1
    
    rolling_mean, rollind_std = rolling(buffer_MAPE_TP)
    dummy_mean = np.mean(rolling_mean)
    MAPE_mean_TP.append(dummy_mean)
    dummy_std = np.mean(rollind_std)
    MAPE_std_TP.append(dummy_std)
    del rolling_mean, rollind_std, buffer_MAPE_TP

    rolling_mean, rollind_std = rolling(buffer_MAPE_BP)
    dummy_mean = np.mean(rolling_mean)
    MAPE_mean_BP.append(dummy_mean)
    dummy_std = np.mean(rollind_std)
    MAPE_std_BP.append(dummy_std)
    del rolling_mean, rollind_std, buffer_MAPE_BP
    
    rolling_mean, rollind_std = rolling(buffer_MAPE_Time)
    dummy_mean = np.mean(rolling_mean)
    MAPE_mean_Time.append(dummy_mean)
    dummy_std = np.mean(rollind_std)
    MAPE_std_Time.append(dummy_std)
    del rolling_mean, rollind_std, buffer_MAPE_Time

"""
infer_time_TP = infer_time_TP / (98525 * 5)
print('Top power Model. Infer time: ' + str(infer_time_TP) + ' Train time: ' + str(train_time_TP))
infer_time_BP = infer_time_BP / (98525 * 5)
print('Bottom power Model. Infer time: ' + str(infer_time_BP) + ' Train time: ' + str(train_time_BP))
infer_time_Time = infer_time_Time / (98525 * 5)
print('Time Model. Infer time: ' + str(infer_time_Time) + ' Train time: ' + str(train_time_Time))
"""

model_type = ["Top-power", "Bottom-power", "Time"]
resultados = {"Top-power": {}, "Bottom-power": {}, "Time": {}}
for mt in model_type:
    resultados[mt] = {"Mape_mean": {}, "Mape_sdv": {}, "Mape_mean_dev": {}, "Infer_time": {}, "Train_time": {}}

resultados["Top-power"]['Mape_mean'] = float(np.mean(MAPE_mean_TP))
resultados["Top-power"]['Mape_sdv'] = float(np.mean(MAPE_std_TP))
resultados["Top-power"]['Mape_mean_dev'] = float(np.std(MAPE_mean_TP))
resultados["Top-power"]['Infer_time'] = infer_time_TP / (98525 * 5)
resultados["Top-power"]['Train_time'] = train_time_TP/5

resultados["Bottom-power"]['Mape_mean'] = float(np.mean(MAPE_mean_BP))
resultados["Bottom-power"]['Mape_sdv'] = float(np.mean(MAPE_std_BP))
resultados["Bottom-power"]['Mape_mean_dev'] = float(np.std(MAPE_mean_BP))
resultados["Bottom-power"]['Infer_time'] = infer_time_BP / (98525 * 5)
resultados["Bottom-power"]['Train_time'] = train_time_BP/5

resultados["Time"]['Mape_mean'] = float(np.mean(MAPE_mean_Time))
resultados["Time"]['Mape_sdv'] = float(np.mean(MAPE_std_Time))
resultados["Time"]['Mape_mean_dev'] = float(np.std(MAPE_mean_Time))
resultados["Time"]['Infer_time'] = infer_time_Time / (98525 * 5)
resultados["Time"]['Train_time'] = train_time_Time/5

with open("river_results.pkl", "wb") as NN_2layers_dict:
    pickle.dump(resultados, NN_2layers_dict)
NN_2layers_dict.close()

El error en la predicción se puede ver de está forma para cada modelo. Ten en cuenta que la metrica empleada es una media de las últimas 1000 predicciones.

In [10]:
model_names = ["PS Power", "PL Power", "Execution Time"]

for model_name, metric in zip(model_names, metrics):
    print(f"{model_name} - MAPE: {round(metric.get(),2)}%")

PS Power - MAPE: 3.01%
PL Power - MAPE: 2.21%
Execution Time - MAPE: 18.62%


### Proceso Completo

Aquí se repiten todos los pasos anteriores seguidos. Puede tardar un rato puesto que en este caso se van a entrenar todos los modelos con el dataset completo.

In [ ]:
import pandas as pd
import river
import sys
from river import metrics, preprocessing, forest

def features_labels_accommodation(features, labels):
        """Perform accomodation on features and labels. Type casting..."""

        features["user"] = float(features["user"])
        features["kernel"] = float(features["kernel"])
        features["idle"] = float(features["idle"])

        features["Main"] = int(features["Main"])
        features["aes"] = int(features["aes"])
        features["bulk"] = int(features["bulk"])
        features["crs"] = int(features["crs"])
        features["kmp"] = int(features["kmp"])
        features["knn"] = int(features["knn"])
        features["merge"] = int(features["merge"])
        features["nw"] = int(features["nw"])
        features["queue"] = int(features["queue"])
        features["stencil2d"] = int(features["stencil2d"])
        features["stencil3d"] = int(features["stencil3d"])
        features["strided"] = int(features["strided"])

        # Get each model label
        labels = [float(labels[key]) for key in labels]

        return features, labels


#
# Dataset
#

# Read dataset
dataset_df = pd.read_pickle("dataset.pkl")

# Extract features
features_df = dataset_df.drop(["Top power", "Bottom power", "Time"], axis=1)

# Extract labels
labels_df = dataset_df[["Top power", "Bottom power", "Time"]]


#
# Model Initialization
#

# Initialize PS power model
top_power_model = (
                river.preprocessing.StandardScaler() |
                river.tree.HoeffdingAdaptiveTreeRegressor(
                    max_depth=100,
                    grace_period=50,
                    model_selector_decay=0.05,
                    seed=42
                )
            )

# Initialize PL power models
bottom_power_model = (
                river.preprocessing.StandardScaler() |
                river.tree.HoeffdingAdaptiveTreeRegressor(
                    max_depth=100,
                    grace_period=50,
                    model_selector_decay=0.05,
                    seed=42
                )
            )

# Initialize Execution time model
time_model = tmp_model = river.forest.ARFRegressor(seed=42, max_features=None, grace_period=50, n_models = 5, max_depth=100, model_selector_decay=0.05)

# Create model MAPE metrics
top_power_mape = river.utils.Rolling(river.metrics.MAPE(), window_size=1000)
bottom_power_mape = river.utils.Rolling(river.metrics.MAPE(), window_size=1000)
time_mape = river.utils.Rolling(river.metrics.MAPE(), window_size=1000)

# List of metrics and models
models = [top_power_model, bottom_power_model, time_model]
metrics = [top_power_mape, bottom_power_mape, time_mape]


#
# Model Train
#

# Loop over the observations
for features, labels in river.stream.iter_pandas(features_df, labels_df, shuffle=False, seed=42):

    # Features and labels accommodation
    features, labels = features_labels_accommodation(features, labels)

    for model, metric, label in zip(models, metrics, labels):
        # Make a prediction
        y_pred = model.predict_one(features)
        # Train the model
        model.learn_one(features, label)
        # Update metric
        metric.update(label, y_pred)

# Print metrics
for model_name, metric in zip(model_names, metrics):
    print(f"{model_name} - MAPE: {round(metric.get(),2)}%")

PS Power - MAPE: 2.6%
PL Power - MAPE: 2.2%
Execution Time - MAPE: 35.55%
